Detailed code that covers data transformation using Amazon SageMaker Data Wrangler, model training, deploying the Hugging Face model to a SageMaker endpoint, setting up API Gateway with AWS Lambda to expose the endpoint, and calling the OpenAI API to feed the data:

Data Transformation with SageMaker Data Wrangler:

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Step 1: Import Data into SageMaker Data Wrangler
s3_input_data = 's3://your-bucket/input.csv'
data_wrangler_input = sagemaker.wrangler.CSV(s3_input_data)

data_wrangler_project_uri = 's3://your-bucket/data_wrangler_project'
data_wrangler_project = sagemaker.wrangler.Project(
    sagemaker_session=sagemaker_session,
    project_id='cardiovascularpredic',
    project_arn='arn:aws:sagemaker:ap-south-1:xxxxxxxx:project/cardiovascularpredic',
    role=role,
    data=data_wrangler_input,
)

data_wrangler_project.save(data_wrangler_project_uri)

# Step 2: Start SageMaker Processing Job to Transform Data
data_wrangler_processor = data_wrangler_project.process(
    output_name='s3://cardiovascularpredict/output/',
    output_format='csv',
    description='Data transformation job using SageMaker Data Wrangler',
    instance_count=1,
    instance_type='ml.m5.large',
)

data_wrangler_processor.run()

data_transformed_output = data_wrangler_processor.outputs[0].destination


Model Training and Deployment with Hugging Face:

In [ ]:
from sagemaker.huggingface import HuggingFace

# Step 3: Train the Hugging Face model
output_dir = 's3://cardiovascularpredict/input.csv'
hf_estimator = HuggingFace(
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    transformers_version='4.10',
    pytorch_version='1.9',
    py_version='py36',
    output_path=output_dir,
    base_job_name='huggingface-cardio-model-training',
    hyperparameters={
        'epochs': 3,
        'train_batch_size': 32,
        'model_name': 'distilbert-base-uncased',
    },
)

training_data = sagemaker.inputs.TrainingInput(
    data_channel=data_transformed_output,
    content_type='text/csv',
)

hf_estimator.fit(inputs={'train': training_data})

# Step 4: Deploy the Hugging Face model to SageMaker endpoint
huggingface_predictor = hf_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

endpoint_name = huggingface_predictor.endpoint_name


Setting up API Gateway with AWS Lambda:

In [ ]:
import boto3

# Step 5: Create an API Gateway REST API
api_gateway_client = boto3.client('apigateway')

rest_api_id = api_gateway_client.create_rest_api(
    name='CardiovascularPredictionAPI',
    description='API for cardiovascular disease prediction',
)['id']

# Step 6: Create a new resource for the API
resource_id = api_gateway_client.create_resource(
    restApiId=rest_api_id,
    parentId=rest_api_id,
    pathPart='predict',
)['id']

# Step 7: Create a POST method for the resource
api_gateway_client.put_method(
    restApiId=rest_api_id,
    resourceId=resource_id,
    httpMethod='POST',
